In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
import requests
import yaml
import json
import os
import pandas as pd
import gzip
import shutil

In [98]:
from gdc.download import api_download_iterative

Config

In [99]:
with open('conf/user_conf.yaml', 'r') as f:
    user_conf = yaml.load(f)

In [100]:
with open('conf/sys_conf.yaml', 'r') as f:
    sys_conf = yaml.load(f)

In [101]:
conf = {**user_conf, **sys_conf}

Paths

In [102]:
slides_metadata_file = os.path.join(conf['data_path'], 'slides_metadata.csv')
rnaseq_metadata_file = os.path.join(conf['data_path'], 'rnaseq_metadata.csv')

In [103]:
slides_path = os.path.join(conf['data_path'], 'slides')
if not os.path.exists(slides_path):
    os.mkdir(slides_path)

In [104]:
rnaseq_path = os.path.join(conf['data_path'], 'rnaseq')

if not os.path.exists(rnaseq_path):
    os.mkdir(rnaseq_path)

# Download Files

Read metadata

In [105]:
slides_df = pd.read_csv(slides_metadata_file, sep='|')
slides_df = slides_df[['file_name', 'file_id', 'file_size', 'sample_id', 'experimental_strategy']]
slides_df.head(3)

,file_name,file_id,file_size,sample_id,experimental_strategy
0,TCGA-2L-AAQM-11A-01-TSA.svs,ddce0ab5-47b7-43c2-8d67-3d50c48d8ae7,60.99,TCGA-2L-AAQM-11A,Tissue Slide
1,TCGA-HZ-A49I-01A-01-TS1.svs,7ec5c9d0-8c22-4e30-bae8-605c3dcfe744,293.72,TCGA-HZ-A49I-01A,Tissue Slide
2,TCGA-3A-A9IZ-01A-01-TS1.svs,265adb6b-913e-490c-8cea-32d67c1440a2,166.95,TCGA-3A-A9IZ-01A,Tissue Slide


In [106]:
rnaseq_df = pd.read_csv(rnaseq_metadata_file, sep='|')
rnaseq_df = rnaseq_df[['file_name', 'file_id', 'file_size', 'sample_id', 'workflow_type']]
rnaseq_df.head(3)

,file_name,file_id,file_size,sample_id,workflow_type
0,TCGA-FB-AAPS-01A_HTSeq-FPKM-UQ.txt.gz,1e0be26b-da47-4e56-926c-9afc259a2bd0,0.53,TCGA-FB-AAPS-01A,HTSeq - FPKM-UQ
1,TCGA-FB-AAQ3-01A_HTSeq-FPKM-UQ.txt.gz,2803abb9-bf21-4b75-a3c9-687c87b75701,0.50,TCGA-FB-AAQ3-01A,HTSeq - FPKM-UQ
2,TCGA-HZ-8005-01A_HTSeq-Counts.txt.gz,4d98a852-c30a-437b-a214-ee81015d674b,0.25,TCGA-HZ-8005-01A,HTSeq - Counts


In [112]:
slides_df

,file_name,file_id,file_size,sample_id,experimental_strategy
1,TCGA-HZ-A49I-01A-01-TS1.svs,7ec5c9d0-8c22-4e30-bae8-605c3dcfe744,293.72,TCGA-HZ-A49I-01A,Tissue Slide
2,TCGA-3A-A9IZ-01A-01-TS1.svs,265adb6b-913e-490c-8cea-32d67c1440a2,166.95,TCGA-3A-A9IZ-01A,Tissue Slide
3,TCGA-IB-7891-01A-01-BS1.svs,8b5ee711-7836-47e9-a96c-5a1dafdccaec,703.26,TCGA-IB-7891-01A,Tissue Slide
4,TCGA-IB-A5ST-01A-01-TSA.svs,8587e7dc-95b2-4a30-8223-664a8ab7bfe6,366.12,TCGA-IB-A5ST-01A,Tissue Slide
7,TCGA-FB-AAQ2-01A-03-TSC.svs,638c0e4c-9419-416c-8355-8ac023d8edb2,304.14,TCGA-FB-AAQ2-01A,Tissue Slide
8,TCGA-H8-A6C1-01A-01-TS1.svs,1cff51cc-2a5c-4996-9895-bfe6f227a0e3,151.85,TCGA-H8-A6C1-01A,Tissue Slide
10,TCGA-HZ-7920-01A-01-TS1.svs,eb2d2ac5-c29d-48b3-b922-893404259e75,61.02,TCGA-HZ-7920-01A,Tissue Slide
11,TCGA-3A-A9IC-01A-01-TS1.svs,b01cdb94-a2be-4c44-ba87-5057912cb74e,348.22,TCGA-3A-A9IC-01A,Tissue Slide
12,TCGA-HZ-8519-01A-01-TS1.svs,63ebeb64-60d4-49c7-b63a-d7556bfe052e,68.78,TCGA-HZ-8519-01A,Tissue Slide
13,TCGA-IB-8126-01A-01-TS1.svs,b0f1ee22-b711-4c53-8d88-c0f96c9fe7e5,59.41,TCGA-IB-8126-01A,Tissue Slide


## Slides

*Keep only the ones with RNASeq related data?*
*Keep only tissue slides?*

Number of files and count

In [28]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Diagnostic Slide,209,244.53
Tissue Slide,257,57.48


In [14]:
rna_seq_samples = rnaseq_df['sample_id'].unique()

In [15]:
slides_df = slides_df[slides_df['sample_id'].isin(rna_seq_samples)]

In [16]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Tissue Slide,209,51.8


In [17]:
# Download only failed
files_downloaded = os.listdir(slides_path)
slides_df = slides_df[-slides_df['file_name'].isin(files_downloaded)]

In [18]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Tissue Slide,48,12.59


In [ ]:
api_download_iterative(slides_df, slides_path, multiprocess=4)

## RNA-Seq

In [119]:
summary = rnaseq_df.groupby('workflow_type').size()
summary.reset_index()

,workflow_type,0
0,HTSeq - Counts,182
1,HTSeq - FPKM,182
2,HTSeq - FPKM-UQ,182


In [32]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
workflow_type,,
HTSeq - Counts,182,0.05
HTSeq - FPKM,182,0.09
HTSeq - FPKM-UQ,182,0.09


In [34]:
slides_samples = slides_df['sample_id'].unique()

In [36]:
rnaseq_df = rnaseq_df[rnaseq_df['sample_id'].isin(slides_samples)]

In [37]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
workflow_type,,
HTSeq - Counts,182,0.05
HTSeq - FPKM,182,0.09
HTSeq - FPKM-UQ,182,0.09


In [39]:
files = api_download_iterative(rnaseq_df, rnaseq_path, multiprocess=8)

['TCGA-HZ-8005-01A_HTSeq-Counts.txt.gz',
 'TCGA-HV-A7OL-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAPQ-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAPS-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-3A-A9IB-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAQ3-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-AAUU-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-H6-A45N-11A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-PZ-A5RE-01A_HTSeq-Counts.txt.gz',
 'TCGA-IB-7891-01A_HTSeq-Counts.txt.gz',
 'TCGA-2L-AAQE-01A_HTSeq-FPKM.txt.gz',
 'TCGA-F2-A7TX-01A_HTSeq-Counts.txt.gz',
 'TCGA-OE-A75W-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-AAUM-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-7649-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-HZ-7289-01A_HTSeq-Counts.txt.gz',
 'TCGA-IB-AAUR-01A_HTSeq-Counts.txt.gz',
 'TCGA-3A-A9IO-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-A7LX-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-A5SP-01A_HTSeq-FPKM.txt.gz',
 'TCGA-2L-AAQM-01A_HTSeq-Counts.txt.gz',
 'TCGA-HZ-8636-01A_HTSeq-FPKM.txt.gz',
 'TCGA-3A-A9I7-01A_HTSeq-Counts.txt.gz',
 'TCGA-3A-A9IH-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-3A-A9IU-01A

In [42]:
def gunzip(source_filepath, dest_filepath):
    with gzip.open(source_filepath, 'rb') as s_file:
        with open(dest_filepath, 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file)

In [51]:
for file_name in files:
    
    source_filepath = os.path.join(rnaseq_path, file_name)
    dest_filepath = re.sub('\.gz$', '', source_filepath)
    
    gunzip(source_filepath, dest_filepath)
    os.remove(source_filepath)